In [13]:
import os
import sqlite3
import hashlib
import google.generativeai as google_genai

import google.auth
import vertexai
import vertexai.generative_models as vertexai_genai
import vertexai.language_models as vertexai_plam2
from vertexai.generative_models import Content, Part

from typing import List, Optional, Tuple, Union

from dotenv import load_dotenv
load_dotenv()

True

# Google AI Studio

In [2]:
# google_genai.configure(api_key=os.getenv('GOOGLE_API_KEY'), transport='rest')
google_genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))

In [4]:
for m in google_genai.list_models():
    print(m.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-pro
models/gemini-pro-vision
models/embedding-001
models/aqa


In [4]:
llm = google_genai.GenerativeModel('gemini-1.0-pro')

In [5]:
llm.generate_content("The quick brown fox jumps over the lazy dog")

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=glm.GenerateContentResponse({'candidates': [{'content': {'parts': [{'text': 'The quick brown fox jumps over the lazy dog.'}], 'role': 'model'}, 'finish_reason': 1, 'index': 0, 'safety_ratings': [{'category': 9, 'probability': 1, 'blocked': False}, {'category': 8, 'probability': 1, 'blocked': False}, {'category': 7, 'probability': 2, 'blocked': False}, {'category': 10, 'probability': 1, 'blocked': False}], 'token_count': 0, 'grounding_attributions': []}], 'prompt_feedback': {'safety_ratings': [{'category': 9, 'probability': 1, 'blocked': False}, {'category': 8, 'probability': 1, 'blocked': False}, {'category': 7, 'probability': 1, 'blocked': False}, {'category': 10, 'probability': 1, 'blocked': False}], 'block_reason': 0}}),
)

In [6]:
# Setup Gemini Model
def setup_gemini_model(
    model_name:str,
    temperature:float,
    max_output_tokens:int,
    top_p:float,
    top_k:int,
    stop_sequence:Optional[List[str]]=None,
) -> google_genai.GenerativeModel:
    
    # Setup Safety Settings
    safety_settings = {
        # HarmCategory.HARM_CATEGORY_DANGEROUS: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        # HarmCategory.HARM_CATEGORY_DEROGATORY: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        # HarmCategory.HARM_CATEGORY_MEDICAL: HarmBlockThreshold.BLOCK_NONE,
        # HarmCategory.HARM_CATEGORY_SEXUAL: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
        # HarmCategory.HARM_CATEGORY_TOXICITY: HarmBlockThreshold.BLOCK_NONE,
        # HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE,
        # HarmCategory.HARM_CATEGORY_VIOLENCE: HarmBlockThreshold.BLOCK_NONE,
    }
    
    # Setup Model Config
    generation_config = google_genai.GenerationConfig(
        candidate_count=1,
        stop_sequences=stop_sequence,
        max_output_tokens=max_output_tokens,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
    )
    
    # Create a new model
    gemini_model = google_genai.GenerativeModel(
        model_name=model_name,
        safety_settings=safety_settings,
        generation_config=generation_config
        )

    return gemini_model

In [7]:
llm = setup_gemini_model(
    model_name='gemini-1.0-pro',
    temperature=0,
    max_output_tokens=1024,
    top_p=1,
    top_k=50,
)

In [8]:
print(llm)

genai.GenerativeModel(
    model_name='models/gemini-1.0-pro',
    generation_config={'candidate_count': 1, 'max_output_tokens': 1024, 'temperature': 0, 'top_p': 1, 'top_k': 50},
    safety_settings={<HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: 10>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_HARASSMENT: 7>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_HATE_SPEECH: 8>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: 9>: <HarmBlockThreshold.BLOCK_NONE: 4>},
    tools=None,
)


In [9]:
llm.count_tokens("The quick brown fox jumps over the lazy dog").total_tokens

9

# VertexAI

In [2]:
credentials, project_id = google.auth.default()
LOCATION = os.getenv('GCP_LOCATION')

vertexai.init(project=os.getenv('GCP_PROJECT'), location=LOCATION, credentials=credentials)

In [3]:
# Setup Gemini Model
def setup_gemini_model(
    model_name:str,
    temperature:float=0,
    max_output_tokens:int=1024,
    top_p:float=1,
    top_k:int=40,
    stop_sequence:Optional[List[str]]=None,
    is_vertexai_model:bool=False,
) -> Union[google_genai.GenerativeModel, vertexai_genai.GenerativeModel]:
    
    if is_vertexai_model:
        from vertexai.generative_models import GenerativeModel, GenerationConfig, HarmCategory, HarmBlockThreshold
    else:
        from google.generativeai import GenerativeModel, GenerationConfig
        from google.generativeai.types import HarmCategory, HarmBlockThreshold
    
    # Setup Safety Settings
    if is_vertexai_model:
        safety_settings = {
            # HarmCategory.HARM_CATEGORY_DANGEROUS: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
            # HarmCategory.HARM_CATEGORY_DEROGATORY: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
            HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
            # HarmCategory.HARM_CATEGORY_MEDICAL: HarmBlockThreshold.BLOCK_NONE,
            # HarmCategory.HARM_CATEGORY_SEXUAL: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
            # HarmCategory.HARM_CATEGORY_TOXICITY: HarmBlockThreshold.BLOCK_NONE,
            # HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE,
            # HarmCategory.HARM_CATEGORY_VIOLENCE: HarmBlockThreshold.BLOCK_NONE,
        }
    else:
        safety_settings = {
            # HarmCategory.HARM_CATEGORY_DANGEROUS: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
            # HarmCategory.HARM_CATEGORY_DEROGATORY: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
            # HarmCategory.HARM_CATEGORY_MEDICAL: HarmBlockThreshold.BLOCK_NONE,
            # HarmCategory.HARM_CATEGORY_SEXUAL: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
            # HarmCategory.HARM_CATEGORY_TOXICITY: HarmBlockThreshold.BLOCK_NONE,
            # HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE,
            # HarmCategory.HARM_CATEGORY_VIOLENCE: HarmBlockThreshold.BLOCK_NONE,
        }
    
    # Setup Model Config
    generation_config = GenerationConfig(
        candidate_count=1,
        stop_sequences=stop_sequence,
        max_output_tokens=max_output_tokens,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
    )
    
    # Create a new model
    gemini_model = GenerativeModel(
        model_name=model_name,
        safety_settings=safety_settings,
        generation_config=generation_config
        )

    return gemini_model

In [4]:
# Setup TextBison Model in VertexAI
def setup_plam2_model(
    model_name:str,
    temperature:float=0,
    max_output_tokens:int=1024,
    top_p:float=1,
    top_k:int=40,
    stop_sequences:Optional[List[str]]=None,
) -> Tuple[vertexai_plam2.TextGenerationModel, dict]:
    
    if "@002" in model_name:
        max_output_tokens = min(max_output_tokens, 1024)
    top_k = min(top_k, 40)
    
    # Load the Model
    palm2_model = vertexai_plam2.TextGenerationModel.from_pretrained(model_name)
    
    # Set Parameters
    parameters = {
        'temperature': temperature,
        'max_output_tokens': max_output_tokens,
        'top_p': top_p,
        'top_k': top_k,
        'stop_sequences': stop_sequences,
    }
    
    return palm2_model, parameters
    

In [5]:
gemini_model = setup_gemini_model('gemini-1.0-pro', is_vertexai_model=False)
gemini_vertexai_model = setup_gemini_model('gemini-1.0-pro-001', is_vertexai_model=True)

## Text Models

In [34]:
gemini_model_output = gemini_model.generate_content("The quick brown fox jumps over the lazy dog.")

In [38]:
gemini_vertexai_model_output = gemini_vertexai_model.generate_content("The quick brown fox jumps over the lazy dog")

In [49]:
gemini_vertexai_model_output.text, gemini_vertexai_model_output.to_dict()['usage_metadata']['prompt_token_count'], gemini_vertexai_model_output.to_dict()['usage_metadata']['candidates_token_count']

('The quick brown fox jumps over the lazy dog.', 9, 10)

In [23]:
text_bison_model, parameters = setup_plam2_model('text-bison')
text_bison_002_model, parameters = setup_plam2_model('text-bison@002')
text_bison_32k_model, parameters = setup_plam2_model('text-bison-32k')
text_bison_32k_002_model, parameters = setup_plam2_model('text-bison-32k@002')
text_unicorn_001_model, parameters = setup_plam2_model('text-unicorn@001')

In [29]:
text_bison_model.predict("The quick brown fox jumps over the lazy dog", **parameters)

 The quick brown fox jumps over the lazy dog is a pangram, which means that it contains all of the letters of the alphabet. It is often used as a test phrase for typewriters and computers.

In [30]:
text_bison_002_model.predict("The quick brown fox jumps over the lazy dog", **parameters)

 The quick brown fox jumps over the lazy dog is a pangram, which means that it contains all of the letters of the alphabet. It is often used as a test phrase for typewriters and computers.

In [31]:
text_bison_32k_model.predict("The quick brown fox jumps over the lazy dog", **parameters)

 The quick brown fox jumps over the lazy dog is a pangram, which is a sentence that contains all of the letters of the alphabet at least once. This particular pangram is often used to test typewriters and printers, as it ensures that all of the keys are working properly.

In [50]:
text_bison_32k_002_model.predict("The quick brown fox jumps over the lazy dog", **parameters).text

' The quick brown fox jumps over the lazy dog is a pangram, which is a sentence that contains all of the letters of the alphabet at least once. This particular pangram is often used to test typewriters and printers, as it ensures that all of the keys are working properly.'

In [33]:
text_unicorn_001_model.predict("The quick brown fox jumps over the lazy dog", **parameters)

The quick brown fox jumps over the lazy dog is a pangram, which means it contains every letter of the alphabet. It is often used to test typewriters and keyboards.

The sentence is also a holoalphabetic sentence, which means it contains every letter of the alphabet in alphabetical order.

The sentence is believed to have originated in the 19th century. It was first published in The Boston Journal in 1885.

The sentence has been used in a variety of contexts, including in advertising, literature, and music.

In advertising, the sentence has been used to promote products such as typewriters, keyboards, and printers.

In literature, the sentence has been used in a variety of works, including novels, short stories, and poems.

In music, the sentence has been used in a variety of songs, including "The Quick Brown Fox" by The Beach Boys and "The Lazy Dog" by The Grateful Dead.

The sentence is a popular example of a pangram and a holoalphabetic sentence. It is a fun and challenging sentence 

## Gemini Chat

In [ ]:
"""You are a "Public Insurance" Underwriter, researching about the insurance risk liabilities and pro-active risk mitigation work done by or associated with given public entity - {{ insured }}. 

Your task is to analysize the given news article for insurance risk liabilities and pro-active risk mitigation for each of the mentioned "Line of Business" (LOB).

<LIST_OF_LOBs>
General Liability: Liability or pro-active measures for events related to Bodily injury from slips, trips, falls, election irregularities, theft, fraud, misappropriation, faulty design, data breaches, sewer back-ups, except Automobile and professional liability.
Employment Practices Liability: Liability or pro-active measures for events/claims related to discrimination, wrongful termination, sexual harassment, retaliation, misconduct, federal act violations and other employment-process-related allegations.
Law Enforcement Liability: Liability or pro-active measures for events/claims related to wrongful death or imprisonment, civil rights violations, false arrest, excessive force, suicides/deaths in custody, misconduct, excessive force, profiling, shootings, illegal police activities, law enforcement pursuit/chase, personal injury or property damage caused by a wrongful act committed by or on behalf of a public entity while conducting law enforcement activities or operations.
Property Liability: Liability or pro-active measures for events/claims related to weather damage, potential for extreme weather, property damage, disrepair, abandoned or vacant, solar panels, buildings, contents, equipment, and other property owned by the public entity.
Auto Liability: Liability or pro-active measures for events/claims related to vehicular crashes/accidents, law enforcement pursuit/chase, commercial auto and commercial general liability exposure of any land motor vehicle, trailer or semi-trailer for travel on public road. Does not include "mobile equipment".
Employee Benefits Liability: Liability or pro-active measures for events/claims related to retirements, retirement savings, healthcare costs, benefits, errors or omissions in the administration of employee benefit programs.
Zoning Liability: Liability or pro-active measures for events/claims related to land use, ordinance, code, etc. violations, lawsuits and/or disputes.
Liquor Liability: Liability or pro-active measures for events/claims related to Liquor compliance, licensing, code issues and/or licenses obtained or plans to serve liquor to the community/host events.
Dam Liability: Liability or pro-active measures for events/claims related to damages, breaches and/or inspections, water supply hinderance, etc. of dams, lapses in the dam's maintenance and inspection protocols.
Inland Marine Liability: Liability or pro-active measures for events/claims related to loss, damage and/or theft of equipment, machinery, products, or other property at a job site, store at a warehouse, or transported over land by train or truck.
Boiler Liability: Liability or pro-active measures for events/claims related to Power outages and surges due to mechanical failure, frozen pipes, and/or severe weather impacting boiler and machinery.
</LIST_OF_LOBs>

The analysis has to be done in a step-by-step manner.
Follow all the given steps to complete the task and output result of each step in requested format.

STEPS:
Step 1:
Check if the article talks about the concerned public entity - {{ insured }} or individuals related to this public entity and not about any other individuals or any other private entity.
If the article is not about the concerned public entity - {{ insured }}, then the analysis is not required and but do complete the next steps.

Step 2:
Read the complete article as an underwriter scrutinizing the public entity - {{ insured }}.
List out all the LOBs relevant (if any) to the events mentioned involving the public entity - {{ insured }}.
The LOBs to be considered should be only from the above-given list of 11 LOBs 
[General Liability, Employment Practices Liability, Law Enforcement Liability, Property Liability, Auto Liability, Employee Benefits Liability, Zoning Liability, Dam Liability, Inland Marine Liability, Boiler Liability].

Step 3:
For each of the LOBs mentioned in Step 2, identify the reasons given in the article for insurance liability exposure and/or steps taken for proactive risk mitigation. 
The output has to be in JSON format, enclosed in <LOB_CLASSIFICATION></LOB_CLASSIFICATION> XML tags.
Use these exact tags, nothing else. Return "{}" in tags if no LOB is identified in Step 2.
Example:
<LOB_CLASSIFICATION>
{
    LOB1 | STRING - Name of LOB: {
        "detailed_reasoning": STRING | Why does the article invokes this particular insurance liability exposure and/or steps taken for proactive risk mitigation. The reasoning should be based on actual incidents mentioned in the article and not what would have happened or can happen,
        "identified": BOOLEAN - Whether LOB risk or risk-mitigation is actually identified,
    },
    LOB2 : {
        "detailed_reasoning": STRING,
        "identified": BOOLEAN,
    }
}
</LOB_CLASSIFICATION>

<ARTICLE>
{{ article }}
<\ARTICLE>

OUTPUT:
"""

In [8]:
insured = "TOWN OF MCCORDSVILLE, HANCOCK COUNTY, INDIANA"

In [6]:
article = """HANCOCK COUNTY — A few weeks into the new year, officials with the Hancock County Sheriff’s Department (HCSD) shared run statistics from 2023 and compared them with numbers from 2022 as well as from 2021.

Captain Robert Harris, public information officer for the Sheriff’s office, said the statistic show 2023 was a busy year for the department, with huge numbers across the board starting with approximately 25,707 total patrol events.

The patrol event numbers from 2023 are up from 2022 when they had 23,956 patrol events, and they’re even higher than the ones recorded in 2021 when they had 24,138 events.

“I believe this is due to growth of new business, residents and commuters in Hancock County,” Harris said.

Sheriff’s officials are on record saying the creation of many large warehouse businesses on the west side of the county kept them moving in 2023 with numerous calls per day.

Growth throughout the county could explain why HCSD field arrests were also up with 730 field arrests in 2023. That’s several hundred more than in 2022 when there were only 521 field arrests compared with only 447 field arrests in 2021.

More growth means more vehicles on county roads, and that means the number of traffic stops in 2023 increased by over 1,400 to 5,061 compared to 2022 numbers when there were only 3,916 traffic stops and 3,016 traffic stops in 2021.

Sheriff’s officials also spend time each year handling situations that end up being civil events. In 2023, they had 2,137 civil runs compared with 1,987 civil runs in 2022. There were 1,803 in 2021.

When it came to incident reports, case reports to investigations, numbers were up there as well. There were 1,466 reports in 2023, up nearly 100 from 2022 when there were 1,351 reports to investigations. Still, the highest incidents for reports to investigation occurred in 2021 when there where 1,515.

The HCSD also keeps track of happenings at the Hancock County Jail. Harris noted jail bookings were also up, but just slightly in 2023. Last year, 2,192 inmates were booked into the new jail. That number is up by only 20 inmates compared to 2022 numbers when 2,172 inmates were booked into the jail. Both years though are way up from 2021 when the new jail wasn’t yet open and only 1,246 people were taken into custody.

Jail release numbers show while many are taken into custody, there are also high turnover numbers. The jail released 2,160 inmates in 2023, up slightly from 2022 when they released 2,090 inmates. Release could include transferring inmates to the Indiana Department of Corrections; another county jail, probation or outright release. Jail releases were 1,197 in 2021.

While the report numbers are up pretty much across the board for Sheriff’s officials, Harris noted the force is doing as best it can to keep up with the higher demands for public service and officials feel like the volume of work will continue to increase in the coming years.

“I feel we will continue to see increased numbers as the county grows,” Harris said.

With the growth, officials at the Sheriff’s office note they will do all they can to continue to patrol and monitor the county as best they can, but with so much growth they’re going to have to hire more deputies both for patrol and at the jail.

“We would still like to add more deputies on the streets and more employees to expand programs,” Harris said."""

In [15]:
messages = [Content(
    role="user",
    parts=[
        Part.from_text(f'''You are a "Public Insurance" Underwriter, researching about the insurance risk liabilities and pro-active risk mitigation work done by or associated with given public entity - { insured }.
The analysis has to be done in a step-by-step manner.
Follow all the given steps to complete the task and output result of each step in requested format.'''),
        Part.from_text(f'''<ARTICLE>
{ article }
<\ARTICLE>'''),
        Part.from_text(f'''Step 1:
Check if the article talks about the concerned public entity - { insured } or individuals related to this public entity and not about any other individuals or any other private entity.
If the article is not about the concerned public entity - { insured }, then the analysis is not required and but do complete the next steps.
'''),
    ]
)]

In [19]:
input_message = Content(
    role="user",
    parts=[
        Part.from_text(f'''You are a "Public Insurance" Underwriter, researching about the insurance risk liabilities and pro-active risk mitigation work done by or associated with given public entity - { insured }.
The analysis has to be done in a step-by-step manner.
Follow all the given steps to complete the task and output result of each step in requested format.'''),
        Part.from_text(f'''<ARTICLE>
{ article }
<\ARTICLE>'''),
        Part.from_text(f'''Step 1:
Check if the article talks about the concerned public entity - { insured } or individuals related to this public entity and not about any other individuals or any other private entity.
If the article is not about the concerned public entity - { insured }, then the analysis is not required and but do complete the next steps.
'''),
    ]
)

In [20]:
chat = gemini_vertexai_model.start_chat(history=[])

In [21]:
chat.history

[]

In [22]:
response = chat.send_message(input_message)

In [23]:
response

candidates {
  content {
    role: "model"
    parts {
      text: "The article is about the Hancock County Sheriff\'s Department (HCSD), which is a public entity in Hancock County, Indiana. The article does not mention the Town of McCordsville, Hancock County, Indiana. Therefore, the analysis is not required."
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
}
usage_metadata {
  prompt_token_count: 1065
  candidates_token_count: 51
  total_token_count: 1116
}

In [25]:
response = chat.send_message(f"""Step 2:
Read the complete article as an underwriter scrutinizing the public entity - { insured }.
List out all the LOBs relevant (if any) to the events mentioned involving the public entity - { insured }.
The LOBs to be considered should be only from the above-given list of 11 LOBs 
[General Liability, Employment Practices Liability, Law Enforcement Liability, Property Liability, Auto Liability, Employee Benefits Liability, Zoning Liability, Dam Liability, Inland Marine Liability, Boiler Liability].
""")

In [26]:
response

candidates {
  content {
    role: "model"
    parts {
      text: "The article does not mention the Town of McCordsville, Hancock County, Indiana. Therefore, I cannot list out the LOBs relevant to the events mentioned involving the public entity - TOWN OF MCCORDSVILLE, HANCOCK COUNTY, INDIANA."
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
}
usage_metadata {
  prompt_token_count: 1242
  candidates_token_count: 49
  total_token_count: 1291
}

In [29]:
response = chat.send_message("""Step 3:
For each of the LOBs mentioned in Step 2, identify the reasons given in the article for insurance liability exposure and/or steps taken for proactive risk mitigation. 
The output has to be in JSON format, enclosed in <LOB_CLASSIFICATION></LOB_CLASSIFICATION> XML tags.
Use these exact tags, nothing else. Return "<LOB_CLASSIFICATION>{}</LOB_CLASSIFICATION>" in tags if no LOB is identified in Step 2.
Example:
<LOB_CLASSIFICATION>
{
    LOB1 | STRING - Name of LOB: {
        "detailed_reasoning": STRING | Why does the article invokes this particular insurance liability exposure and/or steps taken for proactive risk mitigation. The reasoning should be based on actual incidents mentioned in the article and not what would have happened or can happen,
        "identified": BOOLEAN - Whether LOB risk or risk-mitigation is actually identified,
    },
    LOB2 : {
        "detailed_reasoning": STRING,
        "identified": BOOLEAN,
    }
}
</LOB_CLASSIFICATION>""")

In [30]:
response

candidates {
  content {
    role: "model"
    parts {
      text: "<LOB_CLASSIFICATION>{}</LOB_CLASSIFICATION>"
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
}
usage_metadata {
  prompt_token_count: 1747
  candidates_token_count: 13
  total_token_count: 1760
}

In [ ]:
"""The analysis has to be done in a step-by-step manner.
Follow all the given steps to complete the task and output result of each step in requested format.



<ARTICLE>
{{ article }}
<\ARTICLE>


for insurance risk liabilities and pro-active risk mitigation for each of the mentioned "Line of Business" (LOB).


Your task is to analysize the given news article for insurance risk liabilities and pro-active risk mitigation for each of the mentioned "Line of Business" (LOB).

<LIST_OF_LOBs>
General Liability: Liability or pro-active measures for events related to Bodily injury from slips, trips, falls, election irregularities, theft, fraud, misappropriation, faulty design, data breaches, sewer back-ups, except Automobile and professional liability.
Employment Practices Liability: Liability or pro-active measures for events/claims related to discrimination, wrongful termination, sexual harassment, retaliation, misconduct, federal act violations and other employment-process-related allegations.
Law Enforcement Liability: Liability or pro-active measures for events/claims related to wrongful death or imprisonment, civil rights violations, false arrest, excessive force, suicides/deaths in custody, misconduct, excessive force, profiling, shootings, illegal police activities, law enforcement pursuit/chase, personal injury or property damage caused by a wrongful act committed by or on behalf of a public entity while conducting law enforcement activities or operations.
Property Liability: Liability or pro-active measures for events/claims related to weather damage, potential for extreme weather, property damage, disrepair, abandoned or vacant, solar panels, buildings, contents, equipment, and other property owned by the public entity.
Auto Liability: Liability or pro-active measures for events/claims related to vehicular crashes/accidents, law enforcement pursuit/chase, commercial auto and commercial general liability exposure of any land motor vehicle, trailer or semi-trailer for travel on public road. Does not include "mobile equipment".
Employee Benefits Liability: Liability or pro-active measures for events/claims related to retirements, retirement savings, healthcare costs, benefits, errors or omissions in the administration of employee benefit programs.
Zoning Liability: Liability or pro-active measures for events/claims related to land use, ordinance, code, etc. violations, lawsuits and/or disputes.
Liquor Liability: Liability or pro-active measures for events/claims related to Liquor compliance, licensing, code issues and/or licenses obtained or plans to serve liquor to the community/host events.
Dam Liability: Liability or pro-active measures for events/claims related to damages, breaches and/or inspections, water supply hinderance, etc. of dams, lapses in the dam's maintenance and inspection protocols.
Inland Marine Liability: Liability or pro-active measures for events/claims related to loss, damage and/or theft of equipment, machinery, products, or other property at a job site, store at a warehouse, or transported over land by train or truck.
Boiler Liability: Liability or pro-active measures for events/claims related to Power outages and surges due to mechanical failure, frozen pipes, and/or severe weather impacting boiler and machinery.
</LIST_OF_LOBs>


The output has to be in JSON format, enclosed in <LOB_CLASSIFICATION></LOB_CLASSIFICATION> XML tags.
Use these exact tags, nothing else. Return <LOB_CLASSIFICATION>{}</LOB_CLASSIFICATION> in tags if no LOB is identified in Step 2.
"""